In [2]:
import http.client
import json
import pandas as pd
from pandas.io.json import json_normalize
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# month = 4
# day = 14

# conn = http.client.HTTPSConnection("api.sportradar.us")

# conn.request("GET",
#              "/mlb/trial/v6.5/en/games/2019/{0}/{1}/boxscore."
#              "json?api_key=bw84ac36vu34rdk5vkkh4psx".format(month, day))

# res = conn.getresponse()
# data = res.read()

# baseball_data = []
# baseball_data.append(data.decode("utf-8"))

# baseball_json = []
# baseball_json.append(json.loads(baseball_data[0]))
# del baseball_json[0]['_comment']

# baseball_normal = json_normalize(baseball_json)
# baseball_normal = baseball_normal.drop(columns=
#                                        ['league.alias', 'league.date', 'league.id', 'league.name'])

# baseball_normal_14 = json_normalize(baseball_normal.iloc[0, 0])

# with open('outfile_april_14_pre', 'wb') as fp:
#     pickle.dump(baseball_normal_14, fp)

In [3]:
with open('baseball2018', 'rb') as fp:
    baseball_data = pickle.load(fp)  # Load up the 2018 data

del baseball_data[-1]  # Last day, no games
del baseball_data[108]  # All star break, no games
del baseball_data[108]  # All star break, no games
del baseball_data[108]  # All star break, no games
del baseball_data[30]  # April 31 does not exist
del baseball_data[91]  # June 31 does not exist

baseball_json = []

for days in baseball_data:
    baseball_json.append(json.loads(days))

for days in baseball_json:
    del days['_comment']  # API pull had weird last key value pair, delete

baseball_json_norm = json_normalize(baseball_json)
baseball_json_norm = baseball_json_norm.drop(
    columns=['league.alias', 'league.date', 'league.id', 'league.name'])

data = json_normalize(baseball_json_norm.iloc[0, 0])
for rows in range(1, 180):
    data = data.append(json_normalize(baseball_json_norm.iloc[rows, 0]),
                       ignore_index=True, sort=False)

data['home.win.pct'] = data['game.home.win'] / (data['game.home.win'] + data['game.home.loss'])
data['away.win.pct'] = data['game.away.win'] / (data['game.away.win'] + data['game.away.loss'])
data['home.pitcher.win.pct'] = data['game.home.probable_pitcher.win'] / (data['game.home.probable_pitcher.win'] + data['game.home.probable_pitcher.loss'])
data['away.pitcher.win.pct'] = data['game.away.probable_pitcher.win'] / (data['game.away.probable_pitcher.win'] + data['game.away.probable_pitcher.loss'])
data['total.runs'] = data['game.home.runs'] + data['game.away.runs']  # Response variable!

with open('outfile_april_12_pre', 'rb') as fp:
    baseball_data1 = pickle.load(fp)

data_all_cols = list(data.columns.values)
data_good_cols = list(baseball_data1.columns.values)
data_good_cols.append('home.win.pct')
data_good_cols.append('away.win.pct')
data_good_cols.append('total.runs')
data = data.drop(list(set(data_all_cols) - set(data_good_cols)), axis=1)
# data contains only the 67 variables available before today's MLB games start

# Must drop additional variables will not know before game starts
data = data.drop(['game.away.errors', 'game.away.hits', 'game.away.runs',
                  'game.home.errors', 'game.home.hits', 'game.home.runs'], axis=1)

# Must drop unnecessary variables
data = data.drop(['game.away.id', 'game.away.market', 'game.away.name',
                  'game.away.probable_pitcher.first_name',
                  'game.away.probable_pitcher.jersey_number',
                  'game.away.probable_pitcher.last_name',
                  'game.away.probable_pitcher.preferred_name',
                  'game.away_team', 'game.coverage',
                  'game.game_number', 'game.home.id',
                  'game.home.market', 'game.home.name',
                  'game.home.probable_pitcher.first_name',
                  'game.home.probable_pitcher.jersey_number',
                  'game.home.probable_pitcher.last_name',
                  'game.home.probable_pitcher.preferred_name',
                  'game.home_team', 'game.rescheduled',
                  'game.scheduled', 'game.status',
                  'game.venue.address', 'game.venue.country',
                  'game.venue.market', 'game.venue.state',
                  'game.venue.zip', 'game.weather.forecast.obs_time',
                  'game.id', 'game.venue.city', 'game.venue.id',
                  'game.double_header', 'game.venue.location.lat',
                  'game.venue.location.lng', 
                 'game.away.loss', 'game.away.win',
                 'game.home.loss', 'game.home.win',
                 'game.home.probable_pitcher.win',
                 'game.home.probable_pitcher.loss',
                 'game.away.probable_pitcher.win',
                 'game.away.probable_pitcher.loss'], axis=1)    # Might need these: , 
                 #'game.away.probable_pitcher.id',
                 #'game.home.probable_pitcher.id'

data.drop_duplicates(inplace=True)
data = data.drop([373])  # Pesky row

# Pitcher records set to 0 if NaN
# data['game.away.probable_pitcher.loss'].fillna(0, inplace=True)
# data['game.away.probable_pitcher.win'].fillna(0, inplace=True)
# data['game.home.probable_pitcher.loss'].fillna(0, inplace=True)
# data['game.home.probable_pitcher.win'].fillna(0, inplace=True)

# Missing values replaced with "unknown" for text values
data['game.away.probable_pitcher.id'].fillna('unknown', inplace=True)   # Might need to comment out
data['game.home.probable_pitcher.id'].fillna('unknown', inplace=True)    # Might need to comment out
data['game.weather.forecast.condition'].fillna('unknown', inplace=True)
data['game.weather.forecast.wind.direction'].fillna('unknown', inplace=True)
data['game.venue.field_orientation'].fillna('unknown', inplace=True)
data['game.venue.stadium_type'].fillna('unknown', inplace=True)

# Weather data simplified
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate or heavy snow showers", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate or snow showers", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light snow, mist", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("squalls", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("snow, mist", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light sleet", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light drizzle", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy rain possible", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Light rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light rain shower", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light drizzle", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light rain, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light drizzle, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Shower in Vicinity", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("overcast", "Cloudy", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Thundery outbreaks possible", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain shower", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm in vicinity", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("torrential rain shower", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain with thunderstorm, mist", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm, haze", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy rain, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Partly cloudy", "Cloudy", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Patchy rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Rain, Mist", "Mist", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Torrential Rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Moderate or Storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Patchy rain with thunder", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain with storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("storm, rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("storm with storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Mist", "Mist", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Rain with thunder", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Blowing Widespread Dust", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Smoke", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Haze", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Haze, Smoke", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Fog", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Haze, Haze", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Fog", "Mist", case=False)

data.dropna(axis=0, inplace=True)

with open('model_data', 'wb') as fp:
    pickle.dump(data, fp)

In [4]:
with open('outfile_april_14_pre', 'rb') as fp:
    data = pickle.load(fp)  # Load up the 2018 data

data['home.win.pct'] = data['game.home.win'] / (data['game.home.win'] + data['game.home.loss'])
data['away.win.pct'] = data['game.away.win'] / (data['game.away.win'] + data['game.away.loss'])
data['home.pitcher.win.pct'] = data['game.home.probable_pitcher.win'] / (data['game.home.probable_pitcher.win'] + data['game.home.probable_pitcher.loss'])
data['away.pitcher.win.pct'] = data['game.away.probable_pitcher.win'] / (data['game.away.probable_pitcher.win'] + data['game.away.probable_pitcher.loss'])
data['total.runs'] = data['game.home.runs'] + data['game.away.runs']  # Response variable!

with open('outfile_april_12_pre', 'rb') as fp:
    baseball_data1 = pickle.load(fp)

data_all_cols = list(data.columns.values)
data_good_cols = list(baseball_data1.columns.values)
data_good_cols.append('home.win.pct')
data_good_cols.append('away.win.pct')
data_good_cols.append('total.runs')
data = data.drop(list(set(data_all_cols) - set(data_good_cols)), axis=1)
# data contains only the 67 variables available before today's MLB games start

# Must drop additional variables will not know before game starts
data = data.drop(['game.away.errors', 'game.away.hits', 'game.away.runs',
                  'game.home.errors', 'game.home.hits', 'game.home.runs'], axis=1)

# Must drop unnecessary variables
data = data.drop(['game.away.id', 'game.away.market', 'game.away.name',
                  'game.away.probable_pitcher.first_name',
                  'game.away.probable_pitcher.jersey_number',
                  'game.away.probable_pitcher.last_name',
                  'game.away.probable_pitcher.preferred_name',
                  'game.away_team', 'game.coverage',
                  'game.game_number', 'game.home.id',
                  'game.home.market', 'game.home.name',
                  'game.home.probable_pitcher.first_name',
                  'game.home.probable_pitcher.jersey_number',
                  'game.home.probable_pitcher.last_name',
                  'game.home.probable_pitcher.preferred_name',
                  'game.home_team', 
                  'game.scheduled', 'game.status',
                  'game.venue.address', 'game.venue.country',
                  'game.venue.market', 'game.venue.state',
                  'game.venue.zip', 'game.weather.forecast.obs_time',
                  'game.id', 'game.venue.city', 'game.venue.id',
                  'game.double_header', 'game.venue.location.lat',
                  'game.venue.location.lng', 
                 'game.away.loss', 'game.away.win',
                 'game.home.loss', 'game.home.win',
                 'game.home.probable_pitcher.win',
                 'game.home.probable_pitcher.loss',
                 'game.away.probable_pitcher.win',
                 'game.away.probable_pitcher.loss'], axis=1)    # might need to add 'game.rescheduled'

#, 
#                 'game.away.probable_pitcher.id',
#                 'game.home.probable_pitcher.id'

# Pitcher records set to 0 if NaN
# data['game.away.probable_pitcher.loss'].fillna(0, inplace=True)
# data['game.away.probable_pitcher.win'].fillna(0, inplace=True)
# data['game.home.probable_pitcher.loss'].fillna(0, inplace=True)
# data['game.home.probable_pitcher.win'].fillna(0, inplace=True)

# Missing values replaced with "unknown" for text values
data['game.away.probable_pitcher.id'].fillna('unknown', inplace=True) # Might need to comment out
data['game.home.probable_pitcher.id'].fillna('unknown', inplace=True) # Might need to comment out
data['game.weather.forecast.condition'].fillna('unknown', inplace=True)
data['game.weather.forecast.wind.direction'].fillna('unknown', inplace=True)
data['game.venue.field_orientation'].fillna('unknown', inplace=True)
data['game.venue.stadium_type'].fillna('unknown', inplace=True)

# Weather data simplified
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate or heavy snow showers", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate or snow showers", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light snow", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light snow, mist", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("squalls", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("snow, mist", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light sleet", "Snow", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light drizzle", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy rain possible", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Light rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("moderate rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light rain shower", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("patchy light drizzle", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light rain, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("light drizzle, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Shower in Vicinity", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("overcast", "Cloudy", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Thundery outbreaks possible", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain shower", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm in vicinity", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("torrential rain shower", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain with thunderstorm, mist", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("thunderstorm, haze", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("heavy rain, mist", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Partly cloudy", "Cloudy", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Patchy rain", "Rain", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Rain, Mist", "Mist", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Torrential Rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Moderate or Storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Patchy rain with thunder", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("rain with storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("storm, rain", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("storm with storm", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Mist", "Mist", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Rain with thunder", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Blowing Widespread Dust", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Smoke", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Haze", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Haze, Smoke", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Storm, Fog", "Storm", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Haze, Haze", "Haze", case=False)
data['game.weather.forecast.condition'] = data['game.weather.forecast.condition']. \
    str.replace("Fog", "Mist", case=False)

data.dropna(axis=0, inplace=True)

with open('model_predict_data', 'wb') as fp:
    pickle.dump(data, fp)

In [6]:
with open('model_data', 'rb') as fp:
    model_data = pickle.load(fp)  # Load up the 2018 modeling data
    
with open('model_predict_data', 'rb') as fp:
    model_predict_today = pickle.load(fp)  # Load up today' data

model_data = pd.get_dummies(model_data)
model_predict_today = pd.get_dummies(model_predict_today)

# Make sure all columns match

feature_list_2018 = set(list(model_data.columns))
feature_list_today = set(list(model_predict_today.columns))

dropable_columns = list(feature_list_2018.difference(feature_list_today))

model_data = model_data.drop(dropable_columns, axis=1)    

feature_list_2018 = set(list(model_data.columns))
feature_list_today = set(list(model_predict_today.columns))

dropable_columns = list(feature_list_today.difference(feature_list_2018))

model_predict_today = model_predict_today.drop(dropable_columns, axis=1)    

labels = np.array(model_data['total.runs'])
features = model_data.drop('total.runs', axis=1)
feature_list = list(features.columns)
features = np.array(features)

train_features, test_features, train_labels, test_labels = \
    train_test_split(features, labels, test_size=0.15, random_state=99)

forest = RandomForestRegressor(n_estimators=1000, random_state=99)

forest.fit(train_features, train_labels)

predictions = forest.predict(test_features)

errors = abs(predictions - test_labels)

In [7]:
predict_features = model_predict_today.drop('total.runs', axis=1)
predictions_today = forest.predict(predict_features)
today = pd.DataFrame()
today['away'] = data['game.away.abbr']
today['home'] = data['game.home.abbr']
today['predicted.runs'] = predictions_today
today['bookie'] = [7.5,8,7,9,8.5,9,9.5,8,10.5,8]
today['predicted.run.rank'] = today['predicted.runs'].rank()
today['predicted.bookie.rank'] = today['bookie'].rank()
today['the.bet'] = np.where(today['predicted.run.rank'] - today['predicted.bookie.rank'] >= 0, 'OVER', 'UNDER')
today['betting.opportunity'] = abs(today['predicted.run.rank'] - today['predicted.bookie.rank'])

In [8]:
today.sort_values('betting.opportunity', ascending=False)

,away,home,predicted.runs,bookie,predicted.run.rank,predicted.bookie.rank,the.bet,betting.opportunity
13,OAK,TEX,10.865,10.5,4.0,10.0,UNDER,6.0
1,HOU,SEA,13.198,8.0,9.0,4.0,OVER,5.0
2,COL,SF,11.194,7.0,5.0,1.0,OVER,4.0
14,PIT,WSH,12.683,8.0,8.0,4.0,OVER,4.0
8,PHI,MIA,10.528,8.5,3.0,6.0,UNDER,3.0
11,BAL,BOS,12.170,9.5,6.0,9.0,UNDER,3.0
12,TB,TOR,9.331,8.0,1.0,4.0,UNDER,3.0
3,MIL,LAD,13.263,9.0,10.0,7.5,OVER,2.5
10,CWS,NYY,12.313,9.0,7.0,7.5,UNDER,0.5
0,DET,MIN,10.143,7.5,2.0,2.0,OVER,0.0
